In [11]:
import pandas as pd
import altair as alt
import numpy as np



df = pd.read_csv('Marvel_recaudación.csv', sep=';')


df.columns = ['Titulo', 'Anio_Estreno', 'Tipo', 'Recaudacion']


df = df.dropna(subset=['Anio_Estreno', 'Tipo', 'Recaudacion'])
df['Anio_Estreno'] = df['Anio_Estreno'].astype(int)

try:
    df['Recaudacion_USD'] = df['Recaudacion'].astype(float)
except ValueError:

    df['Recaudacion_USD'] = (
        df['Recaudacion']
        .astype(str)
        .str.replace('.', '', regex=False)
        .astype(float)
    )



df['Tipo_Limpio'] = df['Tipo'].str.strip()


df['Recaudacion_MM'] = df['Recaudacion_USD'] / 1_000_000


df['Categoria'] = df['Tipo_Limpio'].replace({
    'Secuela/Precuela': 'Secuela/Precuela',
    'Secuela': 'Secuela/Precuela',
    'Precuela': 'Secuela/Precuela',
    'Original': 'Original'
})


color_scale = alt.Scale(
    domain=['Original', 'Secuela/Precuela'],
    range=['#1c5791', '#D41C16']
)


min_recaudacion_M = df['Recaudacion_MM'].min()
max_recaudacion_M = df['Recaudacion_MM'].max()


tooltip_data = [
    alt.Tooltip('Titulo', title='Película'),
    alt.Tooltip('Anio_Estreno', title='Año de Estreno'),

    alt.Tooltip('Recaudacion_USD', title='Recaudación (USD)', format='$,.0f')
]



chart = alt.Chart(df).mark_circle(size=100).encode(

    x=alt.X('Anio_Estreno:Q', title='Año de Estreno',
            scale=alt.Scale(domain=[df['Anio_Estreno'].min() - 0.5, df['Anio_Estreno'].max() + 0.5]),
            axis=alt.Axis(tickCount=10)),


    y=alt.Y('Recaudacion_MM:Q', title='Recaudación Global (Millones de USD)',

            scale=alt.Scale(domain=[min_recaudacion_M * 0.95, max_recaudacion_M * 1.05]),

            axis=alt.Axis(format='~s', tickCount=5)),


    color=alt.Color('Categoria:N', scale=color_scale, title='Tipo de Película'),


    tooltip=tooltip_data
).properties(
    title='Recaudación de Películas Marvel por Año y Tipo'
).interactive()

chart.display()

alt.Chart(...)